In [139]:
import numpy as np
import pyvista as pv
import os

In [ ]:
# Define mesh path
case_path = '/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/NP_0042/'
mesh_path = case_path + 'mesh/'
geom_path = case_path + 'geom/'
pp_path = case_path + 'data/'
pref = '4-procs/result'
data_path = case_path + pref + '_100.vtu'

# Load mesh
f_vol = pv.read(mesh_path + "fluid/mesh-complete.vtu")
spline = np.load(geom_path + "spline.npy")
contours = np.load(geom_path + "contours.npy")
mesh = pv.read(data_path)

In [218]:
# Create bounding box
bbox = pv.Box((-5.5,5.5,-1.5,1.5,-1.5,1.5))
cell_dimensions = np.array((64,32,32))
bbox_v = bbox.voxelize(dimensions=cell_dimensions + 1)
# bbox = mesh.voxelize(dimensions=cell_dimensions + 1)

# Compute implicit distance
bbox_d = bbox_v.compute_implicit_distance(mesh.extract_surface())
bbox_d['implicit_distance']*=-1
bbox_d = bbox_d.point_data_to_cell_data()
# bbox_m = bbox_d + mesh
# bbox_m = bbox_m.voxelize_binary_mask(dimensions=cell_dimensions + 1,
                                    #  background_value=1,
                                    #  foreground_value=0)
bbox_m = bbox_d.copy()
bbox_m.cell_data['mask'] = bbox_d.cell_data['implicit_distance']>0
bbox_m = bbox_m.point_data_to_cell_data()

b_clip = bbox_d.clip(normal='y',origin=[0,0,0])
b_m_clip = bbox_m.clip(normal='y',origin=[0,0,0])

dist = bbox_d.cell_data['implicit_distance']
mask = bbox_m.cell_data['mask']

# Interpolate results onto query points
data_r = bbox_v.interpolate(mesh)

# Plot
pl = pv.Plotter(shape=(1,2))
pl.subplot(0,0)
# pl.add_mesh(bbox,opacity=0.3,color='lightblue')
# pl.add_mesh(f_vol, style='wireframe',color='lightblue')
pl.add_mesh(b_clip,scalars='implicit_distance',cmap='bwr',clim=[-1,1])
# pl.add_points(spline,color='red')
# for contour in contours:
#     pl.add_points(contour,color='red')
pl.camera_position='zx'
pl.subplot(0,1)
# pl.add_mesh(f_vol, style='wireframe',color='lightblue',opacity=0.3)
pl.add_mesh(b_m_clip,scalars='mask',interpolate_before_map=False,cmap='bwr')
pl.camera_position='zx'
pl.show()

Widget(value='<iframe src="http://localhost:45083/index.html?ui=P_0x76517ab5e510_103&reconnect=auto" class="py…

In [219]:
data_pv = data_r.copy()

scalar_names = ['Pressure', 'Divergence', 'magU', 'u', 'v', 'w', 'magW', 'magWSS']

# Pressure
data_pv['Pressure'] = data_pv.point_data['Pressure']

# Divergence
data_pv['Divergence'] = data_pv.point_data['Divergence']

# Magnitude of velocity
u = data_pv.point_data['Velocity']
magU = np.linalg.norm(u,axis=1)
data_pv['magU'] = magU

# u, v, w
data_pv['u'] = u[:,0]
data_pv['v'] = u[:,1]
data_pv['w'] = u[:,2]

# Magnitude of vorticity
w = data_pv.point_data['Vorticity']
magW = np.linalg.norm(w,axis=1)
data_pv['magW'] = magW

# Magnitude of WSS
WSS = data_pv.point_data['WSS']
magWSS = np.linalg.norm(WSS,axis=1)
data_pv['magWSS'] = magWSS

data_pv = data_pv.point_data_to_cell_data()

for scalar in scalar_names:
    data_pv.cell_data[scalar][mask] = np.nan

In [220]:
# Plot data
pl = pv.Plotter()
pl.add_mesh(data_pv.clip(normal='y',origin=[0,0,0]),scalars = 'magU',clim=[0,20])
pl.camera_position='zx'
pl.show()

Widget(value='<iframe src="http://localhost:45083/index.html?ui=P_0x76514f9755b0_104&reconnect=auto" class="py…

In [240]:
# Assemble coords array
dist = dist.reshape((-1,1))
mask = mask.reshape((-1,1))
coords = np.hstack([bbox_v.cell_centers().points,dist,mask])

# Assemble data array
data = np.dstack([data_pv.cell_data['u'],#.reshape((-1,1)),
                    data_pv.cell_data['v'],#.reshape((-1,1)),
                    data_pv.cell_data['w'],#.reshape((-1,1)),
                    data_pv.cell_data['Pressure']#.reshape((-1,1))
                    ])

# Create directories if they don't exist
os.makedirs(coords_path, exist_ok=True)
np.save(coords_path + 'coords.npy',coords)

data.shape

(1, 65536, 4)

In [ ]:
coords = np.load(data_path + 'coords.npy')
data = np.load(data_path + 'data.npy')

(2, 65536, 4)